In [1]:
import os
import json
import time
import pprint
import pickle
import tweepy

from collections import defaultdict

pp = pprint.PrettyPrinter(indent=4)

## Authentication

In [2]:
# Consumer keys and access tokens, used for OAuth
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

# OAuth process, using the keys and tokens
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Creation of the actual interface, using authentication
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

## Get User Information

In [3]:
pp.pprint(api.me()._json)

{   'contributors_enabled': False,
    'created_at': 'Fri Jul 07 09:20:33 +0000 2017',
    'default_profile': True,
    'default_profile_image': True,
    'description': '',
    'entities': {'description': {'urls': []}},
    'favourites_count': 0,
    'follow_request_sent': False,
    'followers_count': 0,
    'following': False,
    'friends_count': 204,
    'geo_enabled': False,
    'has_extended_profile': False,
    'id': 883254398075322368,
    'id_str': '883254398075322368',
    'is_translation_enabled': False,
    'is_translator': False,
    'lang': 'en',
    'listed_count': 0,
    'location': '',
    'name': 'Digestant',
    'needs_phone_verification': False,
    'notifications': False,
    'profile_background_color': 'F5F8FA',
    'profile_background_image_url': None,
    'profile_background_image_url_https': None,
    'profile_background_tile': False,
    'profile_image_url': 'http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png',
    'profile_image_ur

## Get Friends

In [4]:
friends = []
for friend in tweepy.Cursor(api.friends).items():
    friends.append(friend)

In [10]:
len(friends)

205

In [11]:
# Search user index by screen_name
for i, friend in enumerate(friends):
    if friend._json['screen_name'] == 'TerryUm_ML':
        print(i)
        user_index = i

198


In [12]:
# Print friend object
pp.pprint(vars(friends[user_index]))

{   '_api': <tweepy.api.API object at 0x10fae55f8>,
    '_json': {   'blocked_by': False,
                 'blocking': False,
                 'contributors_enabled': False,
                 'created_at': 'Fri Jan 20 04:46:32 +0000 2017',
                 'default_profile': False,
                 'default_profile_image': False,
                 'description': 'A PhD student at UWaterloo. \n'
                                'Interested in DL for human motion.\n'
                                'Share DL papers on arXiv everyday.\n'
                                'Most-cited DL paper list (2012-2016) '
                                'https://t.co/H2yD8jey3N',
                 'entities': {   'description': {   'urls': [   {   'display_url': 'goo.gl/VEl3g8',
                                                                    'expanded_url': 'https://goo.gl/VEl3g8',
                                                                    'indices': [   136,
                                  

In [13]:
""" Get some useful information """
print(friends[user_index].name)
print(friends[user_index].screen_name)
print(friends[user_index].description)
print(friends[user_index].lang)
pp.pprint(friends[user_index].status._json) # We can use this to check whether a user has new tweet.

Terry Taewoong Um
TerryUm_ML
A PhD student at UWaterloo. 
Interested in DL for human motion.
Share DL papers on arXiv everyday.
Most-cited DL paper list (2012-2016) https://t.co/H2yD8jey3N
en
{   'contributors': None,
    'coordinates': None,
    'created_at': 'Mon Jul 10 21:37:37 +0000 2017',
    'entities': {   'hashtags': [],
                    'symbols': [],
                    'urls': [   {   'display_url': 'twitter.com/i/web/status/8…',
                                    'expanded_url': 'https://twitter.com/i/web/status/884527052652720128',
                                    'indices': [104, 127],
                                    'url': 'https://t.co/oX4XrECpj8'}],
                    'user_mentions': []},
    'favorite_count': 18,
    'favorited': False,
    'geo': None,
    'id': 884527052652720128,
    'id_str': '884527052652720128',
    'in_reply_to_screen_name': None,
    'in_reply_to_status_id': None,
    'in_reply_to_status_id_str': None,
    'in_reply_to_user_id': N

In [14]:
""" Get screen names from all friends
    Language code: https://dev.twitter.com/web/overview/languages
"""
# lang_blacklist = ['ja', 'ru', 'ko']
lang_blacklist = []
[(friend.name, friend.screen_name, friend.lang) for friend in friends if friend.lang not in lang_blacklist]

[('Han Xiao', 'hxiao', 'en'),
 ('酪梨壽司', 'cwyuni', 'en'),
 ('Apache Spark', 'ApacheSpark', 'en'),
 ('holden karau', 'holdenkarau', 'en'),
 ('Matei Zaharia', 'matei_zaharia', 'en'),
 ('Tavis Ormandy', 'taviso', 'en'),
 ('MalwareTech', 'MalwareTechBlog', 'en'),
 ('Metasploit Project', 'metasploit', 'en'),
 ('Payload Security', 'PayloadSecurity', 'de'),
 ('Peter Haas', 'peter_haas', 'en'),
 ('TrainMan', 'Tra1Nman', 'zh-tw'),
 ('Joseph-CHC', 'jspchc', 'zh-tw'),
 ('Hunter ExploitKit', 'hunter_exploit', 'en'),
 ('Colin Hardy', 'cybercdh', 'en-gb'),
 ('Deep Hub', 'Deep_Hub', 'en'),
 ('wLcY', 'x9090', 'en'),
 ('Haifei Li', 'HaifeiLi', 'en'),
 ('Shadowserver', 'Shadowserver', 'en'),
 ('MalwareTechLab', 'MalwareTechLab', 'en-gb'),
 ('Fernando Mercês', 'MercesFernando', 'en'),
 ('Маркус Кот', '0xD09AD0BED182', 'ru'),
 ('Fabian Wosar', 'fwosar', 'en'),
 ('Fyodor Y', 'fygrave', 'en'),
 ('nao_sec', 'nao_sec', 'ja'),
 ('Microsoft MMPC', 'msftmmpc', 'en'),
 ('herrcore', 'herrcore', 'en'),
 ('Robert McA

## Get A Friend's Tweets
- `entities`: https://dev.twitter.com/overview/api/entities-in-twitter-objects

In [15]:
# Make initial request for most recent tweets (200 is the maximum allowed count)
screen_name = 'TerryUm_ML'
unfiltered_tweets = api.user_timeline(screen_name=screen_name, count=200)

In [16]:
len(unfiltered_tweets)

200

In [17]:
# Filter tweets
def filter_tweets(tweets):
    lang_whitelist = ['en']
    no_reply = True

    if no_reply:
        tweets = [tweet for tweet in tweets if tweet.in_reply_to_screen_name is None]

    if lang_whitelist:
        tweets = [tweet for tweet in tweets if tweet.lang in lang_whitelist]

    return tweets

filtered_tweets = filter_tweets(unfiltered_tweets)
len(filtered_tweets)

158

In [26]:
# Print tweet object
pp.pprint(filtered_tweets[0]._json)

{   'contributors': None,
    'coordinates': None,
    'created_at': 'Mon Jul 10 21:37:37 +0000 2017',
    'entities': {   'hashtags': [],
                    'symbols': [],
                    'urls': [   {   'display_url': 'twitter.com/i/web/status/8…',
                                    'expanded_url': 'https://twitter.com/i/web/status/884527052652720128',
                                    'indices': [104, 127],
                                    'url': 'https://t.co/oX4XrECpj8'}],
                    'user_mentions': []},
    'favorite_count': 18,
    'favorited': False,
    'geo': None,
    'id': 884527052652720128,
    'id_str': '884527052652720128',
    'in_reply_to_screen_name': None,
    'in_reply_to_status_id': None,
    'in_reply_to_status_id_str': None,
    'in_reply_to_user_id': None,
    'in_reply_to_user_id_str': None,
    'is_quote_status': False,
    'lang': 'en',
    'place': None,
    'possibly_sensitive': False,
    'retweet_count': 7,
    'retweeted': False,
  

## Useful data of a tweet

In [11]:
# Get useful infomation from tweet object
def print_tweet(tweet):
    print('* created_at: {}'.format(tweet.created_at))
    print('* id: {}'.format(tweet.id))
    print('* lang: {}'.format(tweet.lang))
    print('* user name: {}'.format(tweet.user.name))
    print('* user screen name: {}'.format(tweet.user.screen_name))
    print('* user profile url: {}'.format(tweet.user.profile_image_url_https))
    print('* favorite_count: {}'.format(tweet.favorite_count))
    print('* retweet count: {}'.format(tweet.retweet_count))
    print('* text: {}'.format(tweet.text))
    print('* hashtags: {}'.format(tweet.entities.get('hashtags', [])))
    print('* symbols: {}'.format(tweet.entities.get('symbols', [])))
    print('* urls: {}'.format(tweet.entities.get('urls', [])))
    print('* user_mentions: {}'.format(tweet.entities.get('user_mentions', [])))
    print('-'*116)

In [ ]:
for tweet in filtered_tweets:
    print_tweet(tweet)

## Save Tweets as Pickle File

### Create folder for every friends

In [5]:
# Make dataset folder
dataset_folder_path = 'dataset/twitter/'
if not os.path.exists(dataset_folder_path):
    os.makedirs(dataset_folder_path)

In [ ]:
# Create folder for each friend
for friend in friends:
    # Make folder
    friend_folder_path = os.path.join(dataset_folder_path, friend.screen_name) 
    if not os.path.exists(friend_folder_path):
        os.makedirs(friend_folder_path)

### Crawl most recent 200 tweets from every friends and save to their corresponding folders

In [29]:
len(friends)

205

In [47]:
%%time

""" Each friend need 3 seconds to retrieve 200 most recent tweets. 
    We have 205 friends currently, so it needs about 10 minutes to process.
"""
def get_friends_tweets():
    
    friends_tweets = {friend.screen_name: {} for friend in friends}
    
    # For every friend, get their recent 200 tweets
    for i, friend in enumerate(friends):
        
        print('* Progess: ({}/{})'.format(i, len(friends)))
        print('* Retrieving {} ({})\'s tweets...'.format(friend.name, friend.screen_name))

        ###################################### 
        ## Get tweets by screen name
        ###################################### 
        tweets = api.user_timeline(screen_name=friend.screen_name, count=200)

        ###################################### 
        ## Segment tweets by date 
        ######################################
        print('* Segmenting {} ({})\'s tweets by date...'.format(friend.name, friend.screen_name))

        tweets_keyed_by_date = defaultdict(lambda: [])

        for tweet in tweets:
            date = tweet.created_at.strftime('%Y-%m-%d')
            tweets_keyed_by_date[date].append(tweet)
            
        # Store date-segemented tweets to the corresponding friend
        friends_tweets[friend.screen_name] = tweets_keyed_by_date
        
        print('-'*116)
        
    return friends_tweets

friends_tweets = get_friends_tweets()

* Process: (0/205)
* User timeline api rate limit: (895/900)
* Retrieving Han Xiao (hxiao)'s tweets...
* Segmenting Han Xiao (hxiao)'s tweets by date...
--------------------------------------------------------------------------------------------------------------------
* Process: (1/205)
* User timeline api rate limit: (894/900)
* Retrieving 酪梨壽司 (cwyuni)'s tweets...
* Segmenting 酪梨壽司 (cwyuni)'s tweets by date...
--------------------------------------------------------------------------------------------------------------------
* Process: (2/205)
* User timeline api rate limit: (893/900)
* Retrieving Apache Spark (ApacheSpark)'s tweets...
* Segmenting Apache Spark (ApacheSpark)'s tweets by date...
--------------------------------------------------------------------------------------------------------------------
* Process: (3/205)
* User timeline api rate limit: (892/900)
* Retrieving holden karau (holdenkarau)'s tweets...
* Segmenting holden karau (holdenkarau)'s tweets by date...
---

In [60]:
def save_friends_tweets(friends_tweets):

    for friend in friends:
        
        tweets_keyed_by_date = friends_tweets.get(friend.screen_name, None)
        
        if tweets_keyed_by_date is None:
            continue
    
        # Make folder for friends
        friend_folder_path = os.path.join(dataset_folder_path, friend.screen_name)
        if not os.path.exists(friend_folder_path):
            os.makedirs(friend_folder_path)
        
        print('* Saving {} ({})\'s tweets...'.format(friend.name, friend.screen_name))
        
        for date, tweets in tweets_keyed_by_date.items():
            # Save to pickle
            tweets_file_path = os.path.join(friend_folder_path, '{}.pkl'.format(date)) 
            with open(tweets_file_path, 'wb') as f:
                # print('  - Writing to {} tweets to "{}"...'.format(len(tweets), tweets_file_path))
                pickle.dump(tweets, f)

        print('-'*116)

save_friends_tweets(friends_tweets)

* Saving Han Xiao (TerryUm_ML)'s tweets...
  - Writing to 1 tweets to "dataset/twitter/hxiao/2017-07-09.pkl"...
  - Writing to 1 tweets to "dataset/twitter/hxiao/2017-05-21.pkl"...
  - Writing to 1 tweets to "dataset/twitter/hxiao/2015-12-10.pkl"...
  - Writing to 1 tweets to "dataset/twitter/hxiao/2015-10-05.pkl"...
  - Writing to 2 tweets to "dataset/twitter/hxiao/2015-10-02.pkl"...
  - Writing to 1 tweets to "dataset/twitter/hxiao/2015-07-15.pkl"...
  - Writing to 1 tweets to "dataset/twitter/hxiao/2015-05-13.pkl"...
  - Writing to 1 tweets to "dataset/twitter/hxiao/2015-03-31.pkl"...
  - Writing to 1 tweets to "dataset/twitter/hxiao/2015-02-06.pkl"...
  - Writing to 1 tweets to "dataset/twitter/hxiao/2015-01-28.pkl"...
  - Writing to 9 tweets to "dataset/twitter/hxiao/2015-01-26.pkl"...
  - Writing to 1 tweets to "dataset/twitter/hxiao/2014-12-25.pkl"...
  - Writing to 2 tweets to "dataset/twitter/hxiao/2014-12-15.pkl"...
  - Writing to 1 tweets to "dataset/twitter/hxiao/2014-11-19

## Read pickle files and restore tweets

In [104]:
def restore_friends_tweets():

    friends_tweets = {screen_name: {} for screen_name in os.listdir(dataset_folder_path)}

    for screen_name in list(friends_tweets.keys()):
        
        print('* Restoring {}\'s tweets...'.format(screen_name))
        
        friend_folder_path = os.path.join(dataset_folder_path, screen_name)
            
        for file_name in os.listdir(friend_folder_path):
            # Read pickle, i.e. tweets_keyed_by_date
            tweets_file_path = os.path.join(friend_folder_path, file_name)
            
            with open(tweets_file_path, 'rb') as f:
                # print('* Reading tweets from "{}"...'.format(tweets_file_path))
                date = file_name.split('.')[0]
                friends_tweets[screen_name][date] = pickle.load(f)
                
        print('-'*116)
        
    return friends_tweets

friends_tweets = restore_friends_tweets()

* Restoring 0x009AD6_810's tweets...
--------------------------------------------------------------------------------------------------------------------
* Restoring 0xD09AD0BED182's tweets...
--------------------------------------------------------------------------------------------------------------------
* Restoring ___OD___'s tweets...
--------------------------------------------------------------------------------------------------------------------
* Restoring __MHSC's tweets...
--------------------------------------------------------------------------------------------------------------------
* Restoring _clem1's tweets...
--------------------------------------------------------------------------------------------------------------------
* Restoring aelsmartin's tweets...
--------------------------------------------------------------------------------------------------------------------
* Restoring AlexRoseJo's tweets...
---------------------------------------------------------

### Test if restore successfully

In [105]:
friends_tweets['TerryUm_ML'].keys()

dict_keys(['2017-06-05', '2017-06-06', '2017-06-08', '2017-06-09', '2017-06-10', '2017-06-11', '2017-06-12', '2017-06-13', '2017-06-15', '2017-06-16', '2017-06-17', '2017-06-19', '2017-06-20', '2017-06-21', '2017-06-22', '2017-06-23', '2017-06-25', '2017-06-26', '2017-06-27', '2017-06-28', '2017-06-29', '2017-06-30', '2017-07-01', '2017-07-02', '2017-07-03', '2017-07-05', '2017-07-06', '2017-07-07', '2017-07-09', '2017-07-10'])

In [108]:
""" Test if we restore tweets successfully """
for tweet in friends_tweets['TerryUm_ML']['2017-07-10']:
    print_tweet(tweet)

* created_at: 2017-07-10 21:37:37
* id: 884527052652720128
* lang: en
* user name: Terry Taewoong Um
* user screen name: TerryUm_ML
* user profile url: https://pbs.twimg.com/profile_images/839986509767954437/wGy9rI9c_normal.jpg
* favorite_count: 19
* retweet count: 7
* text: "An overview of gradient descent optimization algorithms": SGD, Adagrad, Adadelta, RMSprop, Adam, etc.… https://t.co/oX4XrECpj8
* hashtags: []
* symbols: []
* urls: [{'url': 'https://t.co/oX4XrECpj8', 'expanded_url': 'https://twitter.com/i/web/status/884527052652720128', 'display_url': 'twitter.com/i/web/status/8…', 'indices': [104, 127]}]
* user_mentions: []
--------------------------------------------------------------------------------------------------------------------
* created_at: 2017-07-10 21:30:32
* id: 884525268278398979
* lang: en
* user name: Terry Taewoong Um
* user screen name: TerryUm_ML
* user profile url: https://pbs.twimg.com/profile_images/839986509767954437/wGy9rI9c_normal.jpg
* favorite_count:

## Check API Rate Limit Status
- Rate limit table: https://dev.twitter.com/rest/public/rate-limits

In [24]:
rate_limit_status = api.rate_limit_status()

print('friends:')
print('limit', rate_limit_status['resources']['friends']['/friends/list']['limit'])
print('remaining', rate_limit_status['resources']['friends']['/friends/list']['remaining'])
print('reset', time.ctime(rate_limit_status['resources']['friends']['/friends/list']['reset']))
print()
print('user timeline:')
print('limit', rate_limit_status['resources']['statuses']['/statuses/user_timeline']['limit'])
print('remaining', rate_limit_status['resources']['statuses']['/statuses/user_timeline']['remaining'])
print('reset', time.ctime(rate_limit_status['resources']['statuses']['/statuses/user_timeline']['reset']))

friends:
limit 15
remaining 0
reset Sun Jul 16 15:13:38 2017

user timeline:
limit 900
remaining 860
reset Sun Jul 16 15:13:40 2017
